In [1]:
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd


import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [23]:
data_file = pd.read_csv("../TAMU_FINAL_DATASET_2018.csv")

In [144]:
# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

In [145]:
df.shape

(100000, 469)

In [142]:
sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)

print()
df = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)
# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(2726, 469)
(2726, 469)

(5452, 469)
(4044, 469)
(4044, 468)
(4044,)


In [6]:
# modeling 

x = tf.placeholder(dtype = tf.float32, shape = [None, 468])
y = tf.placeholder(dtype = tf.int32, shape = [None])


# tanh, elu, selu, softplus, and softsign

#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.relu)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.tanh)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.selu)


#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.softplus)
logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.elu)


loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_pred = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("x: ", x)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", correct_pred)

x:  Tensor("Placeholder:0", shape=(?, 468), dtype=float32)
logits:  Tensor("fully_connected/Elu:0", shape=(?, 128), dtype=float32)
loss:  Tensor("Mean:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [7]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
        #print('EPOCH', i)
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: X_train, y: y_train})
        if i % 50 == 0:
            print("Loss: ", loss)
        #print('DONE WITH EPOCH')

Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)


In [8]:

predicted = sess.run([correct_pred], feed_dict={x: X_train})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_train, predicted)])

# Calculate the accuracy
accuracy = match_count / len(X_train)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.485


In [9]:


predicted = sess.run([correct_pred], feed_dict={x: X_test})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])

# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.515


In [10]:
# try some other sklearn model here

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)



pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1.0
0.5834586466165413


In [11]:

from sklearn.svm import SVC

clf = SVC(kernel='rbf', gamma='auto')
clf.fit(X_train, y_train)

print('finish fitting data')


pred_train = clf.predict(X_train)

pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))



finish fitting data
1.0
0.4857142857142857


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(n_estimators=100, max_depth=2, criterion='entropy', random_state=0)
clf.fit(X_train, y_train)



print(clf.feature_importances_)


pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

[0.00374554 0.00859602 0.         0.0339105  0.05324008 0.
 0.         0.         0.0014445  0.         0.         0.
 0.00292562 0.00081044 0.00426018 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.00071658 0.00255474 0.00098884 0.         0.00077617 0.
 0.         0.00169805 0.         0.         0.         0.00141262
 0.01717393 0.00461231 0.00300986 0.00567186 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.0008785  0.00309341 0.         0.
 0.         0.         0.         0.0096721  0.01314899 0.
 0.         0.00362936 0.00085036 0.         0.00724292 0.00070613
 0.         0.         0.00782428 0.         0.         0.
 0.00314188 0.         0.         0.         0.00569966 0.
 0.         0.00893961 0.         0.         0.00073345 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00060753 0.00227425 0.    

In [28]:
important = clf.feature_importances_
extractedIdx = np.nonzero(important)

In [29]:
print(len(extractedIdx[0]))
extractedIdx

109


(array([  0,   1,   3,   4,   8,  12,  13,  14,  24,  25,  26,  28,  31,
         35,  36,  37,  38,  39,  50,  51,  57,  58,  61,  62,  64,  65,
         68,  72,  76,  79,  82,  97,  98, 103, 105, 134, 139, 142, 143,
        145, 146, 149, 152, 154, 158, 159, 165, 169, 170, 174, 175, 176,
        208, 220, 221, 223, 288, 289, 291, 293, 294, 297, 298, 299, 300,
        301, 303, 304, 305, 306, 307, 312, 319, 321, 322, 323, 328, 332,
        337, 341, 347, 358, 378, 379, 380, 381, 382, 383, 384, 387, 388,
        390, 391, 393, 395, 396, 397, 400, 403, 404, 414, 420, 424, 426,
        428, 436, 437, 442, 458]),)

In [21]:
# previously
print(len(extractedIdx[0]))
extractedIdx

107


(array([  0,   1,   2,   3,   4,   7,   8,  10,  12,  15,  25,  26,  35,
         36,  37,  39,  40,  46,  55,  56,  57,  58,  62,  63,  64,  65,
         66,  67,  68,  69,  79,  82,  93,  97,  98, 100, 105, 118, 134,
        139, 142, 143, 145, 146, 148, 149, 151, 152, 154, 158, 159, 161,
        170, 174, 175, 219, 221, 291, 296, 297, 298, 299, 300, 301, 304,
        305, 306, 307, 310, 312, 313, 319, 322, 323, 332, 334, 336, 346,
        378, 379, 380, 381, 382, 383, 387, 388, 390, 391, 393, 395, 396,
        397, 403, 409, 410, 412, 414, 416, 417, 419, 423, 424, 426, 434,
        436, 437, 458]),)

In [56]:
print(df.columns[0])
extracted_name = []
for idx in extractedIdx:
    extracted_name.append(df.columns[idx])

print(len(extracted_name[0]))
lst = list(extracted_name[0])
newDF = df[lst].copy()

ID
109


In [57]:
print(newDF.columns)

Index(['ID', 'AGE', 'ORIG_REAS_ENTITLE_CD', 'RECON_MA_RISK_SCORE_NBR',
       'CON_VISIT_04_Q03', 'CON_VISIT_21_Q03', 'CON_VISIT_03_Q02',
       'CON_VISIT_03_Q04', 'CON_VISIT_30_Q02', 'CON_VISIT_30_Q04',
       ...
       'Length_residence', 'Est_Net_worth', 'CDC', 'CDC_EYE_GAP', 'BCS_GAP',
       'COL_GAP', 'STATIN_ELIG', 'SEX_CD_F', 'HOSPICE_IND_Y',
       'Diab_Type_Diabetes Type I'],
      dtype='object', length=109)


In [ ]:
# CON_VISIT 11, 22; POT_VISIT_65_Q02 - Q03; 
# Population_density_centile_US; RENAL; Diab_Type_Diabetes Type I; 
# DIABETES
newDF['']

In [58]:
newDF.describe()

,ID,AGE,ORIG_REAS_ENTITLE_CD,RECON_MA_RISK_SCORE_NBR,CON_VISIT_04_Q03,CON_VISIT_21_Q03,CON_VISIT_03_Q02,CON_VISIT_03_Q04,CON_VISIT_30_Q02,CON_VISIT_30_Q04,...,Length_residence,Est_Net_worth,CDC,CDC_EYE_GAP,BCS_GAP,COL_GAP,STATIN_ELIG,SEX_CD_F,HOSPICE_IND_Y,Diab_Type_Diabetes Type I
count,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,...,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.000000,4023.00000,4023.000000,4023.000000
mean,50217.926423,74.544370,0.279891,1.378850,0.534178,0.891872,0.276162,0.282376,0.121800,0.103157,...,16.138454,211311.210539,0.239125,0.103903,0.051454,0.160825,0.478747,0.52349,0.003231,0.037534
std,28973.357908,9.446673,0.450107,1.103054,1.715684,2.597763,1.623744,1.281706,1.220201,0.838558,...,17.629156,275881.935722,0.426602,0.305172,0.220950,0.367415,0.499610,0.49951,0.056761,0.190090
min,9.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,24980.000000,69.000000,0.000000,0.631500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,50330.000000,74.000000,0.000000,1.034000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.000000,87500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000
75%,75125.500000,81.000000,1.000000,1.759500,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,19.000000,312500.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.00000,0.000000,0.000000
max,99960.000000,95.000000,2.000000,8.295000,38.000000,34.000000,43.000000,21.000000,60.000000,27.000000,...,99.000000,1000000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [59]:
newDF.shape

(4023, 109)

In [53]:
df.shape

(4023, 469)

In [44]:
len(y)

4023

In [62]:


newDF['AMI_FLAG'] = pd.Series(y, index=newDF.index)
# use only the 107 features, retrain models
df = newDF
print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(4023, 110)
(4023, 110)
(4023, 109)
(4023,)


In [65]:
df.shape[1]

110

In [66]:
# modeling 

x = tf.placeholder(dtype = tf.float32, shape = [None, (df.shape[1]-1)])
y = tf.placeholder(dtype = tf.int32, shape = [None])


# tanh, elu, selu, softplus, and softsign

#logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.elu)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.tanh)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.selu)


# this two model has better accuracy -------------

logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.relu)
logits = tf.contrib.layers.fully_connected(x, 50, tf.nn.softplus)



loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_pred = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("x: ", x)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", correct_pred)

x:  Tensor("Placeholder_2:0", shape=(?, 109), dtype=float32)
logits:  Tensor("fully_connected_2/Softplus:0", shape=(?, 50), dtype=float32)
loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax_1:0", shape=(?,), dtype=int64)


In [67]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
        #print('EPOCH', i)
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: X_train, y: y_train})
        if i % 50 == 0:
            print("Loss: ", loss)
        #print('DONE WITH EPOCH')

Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)
Loss:  Tensor("Mean_2:0", shape=(), dtype=float32)


In [68]:

predicted = sess.run([correct_pred], feed_dict={x: X_train})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_train, predicted)])

# Calculate the accuracy
accuracy = match_count / len(X_train)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.509


In [69]:


predicted = sess.run([correct_pred], feed_dict={x: X_test})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])

# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.501


In [70]:



clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)



pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1.0
0.5662650602409639


In [71]:
clf = SVC(kernel='rbf', gamma='auto')
clf.fit(X_train, y_train)

print('finish fitting data')


pred_train = clf.predict(X_train)

pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

finish fitting data
1.0
0.48268072289156627


In [72]:

clf = RandomForestClassifier(n_estimators=100, max_depth=2, criterion='entropy', random_state=0)
clf.fit(X_train, y_train)



print(clf.feature_importances_)


pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

[0.00238801 0.0543412  0.         0.09734659 0.         0.00043926
 0.         0.00333447 0.         0.         0.0002895  0.00098941
 0.         0.00385695 0.00616293 0.01821779 0.00362206 0.00901828
 0.00144248 0.00059018 0.00079547 0.02541101 0.00031664 0.00029459
 0.00248738 0.00193534 0.00514849 0.         0.00103364 0.
 0.00071149 0.         0.00050426 0.         0.00083876 0.00170695
 0.0077444  0.00071195 0.00908005 0.00132187 0.00832242 0.02381241
 0.0071825  0.         0.         0.0192523  0.00179057 0.0009103
 0.         0.         0.01058999 0.00081268 0.         0.
 0.00037833 0.         0.         0.00060358 0.00233617 0.00061518
 0.         0.0047635  0.00899555 0.02028387 0.         0.00156675
 0.0014624  0.00067006 0.00081965 0.01857982 0.00284107 0.
 0.00062783 0.         0.00081742 0.02008973 0.00054156 0.
 0.00025157 0.         0.         0.         0.         0.16118625
 0.06384068 0.09157049 0.02218334 0.05616918 0.04684235 0.
 0.04143103 0.00297697 0.02590653 0.

In [78]:
y_data = df['AMI_FLAG']

In [80]:
len(y_data)

4023

In [83]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X, y_data, cv = 5)

In [84]:
scores

array([0.43796526, 0.66583851, 0.6641791 , 0.66169154, 0.39552239])

In [105]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [200, 300, 400, 500, 600]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [10, 20, 30]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]



# crterion selecton
criterion = ['gini', 'entropy']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'criterion': criterion}

print(random_grid)

{'n_estimators': [200, 300, 400, 500, 600], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']}


In [106]:
clf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

In [107]:
rf_random.get_params

<bound method BaseEstimator.get_params of RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 300, 400, 500, 600], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)>

In [108]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, criterion=gini 
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, criterion=gini 
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, criterion=gini 
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy 
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy 
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, criterion=entropy 
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=entropy 
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=entropy 
[CV]  n_es

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.6s


[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=entropy, total=   4.3s
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=entropy, total=   4.3s
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=entropy, total=   3.1s
[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, criterion=entropy 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=entropy, total=   3.2s
[CV] n_estimators=600, min_samples_split=2, min_samples_leaf=1, max_features=log2, max_depth=10, criterion=entropy 
[CV] 

[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy, total=   3.3s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=10, criterion=gini, total=   1.2s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=10, criterion=gini, total=   1.2s
[CV] n_estimators=500, min_samples_split=10, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=gini 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy, total=   3.5s
[CV] n_estimators=500

[CV]  n_estimators=600, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, criterion=entropy, total=   4.2s
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   1.5s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini 
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   1.5s
[CV] n_estimators=600, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=20, criterion=gini 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=gini, total=   1.5s
[CV] n_estimators=600, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=20, criterion=gini 
[CV]  n_estimators=600, min_sam

...

[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=entropy, total=   1.5s
[CV] n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=entropy, total=   1.5s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, criterion=entropy, total=   1.5s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini 
[CV]  n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=30, criterion=gini, total=   2.0s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini 
[CV]  n_estimators=400, 

[CV] n_estimators=300, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=20, criterion=entropy 
[CV]  n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini, total=   2.3s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  n_estimators=400, min_samples_split=2, min_samples_leaf=2, max_features=log2, max_depth=10, criterion=gini, total=   2.4s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=20, criterion=entropy, total=   2.9s
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini 
[CV]  n_estimators=300, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=20, criterion=entropy, total=   2.8s
[CV] n_estimators=500, m

[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini, total=   1.1s
[CV] n_estimators=400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=gini, total=   1.1s
[CV] n_estimators=400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy 
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=None, criterion=gini, total=   3.5s
[CV] n_estimators=400, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=30, criterion=entropy 
[CV]  n_estimators=600, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=None, criterion=gini, total=   3.5s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=None, criterion=gini 
[CV]  n_estimators

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 300, 400, 500, 600], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [109]:
rf_random.best_params_

{'n_estimators': 300,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 10,
 'criterion': 'entropy'}

In [110]:
rf_random.best_params_['n_estimators']

300

In [94]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [95]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

    return accuracy

In [111]:
clf = RandomForestClassifier(n_estimators = rf_random.best_params_['n_estimators'], 
                                    min_samples_leaf= rf_random.best_params_['min_samples_leaf'], 
                                    min_samples_split= rf_random.best_params_['min_samples_split'], 
                                    max_depth = rf_random.best_params_['max_depth'], 
                                    max_features = rf_random.best_params_['max_features'],
                                    criterion = rf_random.best_params_['criterion'],
                                    bootstrap = True,
                                    random_state = 42)

In [113]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [114]:
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.7974025974025974
0.6521084337349398


Compare two training sets result:

In [166]:

# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)

print()
df = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)
# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(2726, 469)
(2726, 469)

(5452, 469)
(4006, 469)
(4006, 468)
(4006,)


In [167]:
print(len(extracted_name[0]))
lst = list(extracted_name[0])
newDF = df[lst].copy()

109


In [168]:
newDF['AMI_FLAG'] = pd.Series(y, index=newDF.index)
# use only the 107 features, retrain models
df = newDF
print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(4006, 110)
(4006, 110)
(4006, 109)
(4006,)


In [169]:
half_clf = RandomForestClassifier(n_estimators = rf_random.best_params_['n_estimators'], 
                                    min_samples_leaf= rf_random.best_params_['min_samples_leaf'], 
                                    min_samples_split= rf_random.best_params_['min_samples_split'], 
                                    max_depth = rf_random.best_params_['max_depth'], 
                                    max_features = rf_random.best_params_['max_features'],
                                    criterion = rf_random.best_params_['criterion'],
                                    bootstrap = True,
                                    random_state = 42)

In [170]:
half_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [171]:
pred_train = half_clf.predict(X_train)
pred_test = half_clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.7999254843517138
0.6580937972768532


In [193]:
# test on the whole dataset

origDF = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
origDF = pd.concat([origDF, categorical_df], axis=1)

In [194]:
origDF = origDF.dropna()

origDF.shape

(74762, 469)

In [195]:
# define the target we would like to predict 
orig_y = origDF['AMI_FLAG']
orig_X = origDF.loc[:, df.columns != 'AMI_FLAG']

print(origDF.columns[0])
print(len(extracted_name[0]))
lst = list(extracted_name[0])
origDF = origDF[lst].copy()

ID
109


In [196]:
origDF.shape

(74762, 109)

In [197]:
orig_y.shape

(74762,)

In [198]:
orig_X.shape

(74762, 109)

In [199]:
whole_X_train, whole_X_test, whole_y_train, whole_y_test = train_test_split(orig_X, orig_y, test_size = 0.33, random_state=42)

In [200]:
clf = RandomForestClassifier(n_estimators = rf_random.best_params_['n_estimators'], 
                                    min_samples_leaf= rf_random.best_params_['min_samples_leaf'], 
                                    min_samples_split= rf_random.best_params_['min_samples_split'], 
                                    max_depth = rf_random.best_params_['max_depth'], 
                                    max_features = rf_random.best_params_['max_features'],
                                    criterion = rf_random.best_params_['criterion'],
                                    bootstrap = True,
                                    random_state = 42)

clf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)>

In [201]:
clf.fit(whole_X_train, whole_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [186]:
pred_train = clf.predict(whole_X_train)
pred_test = clf.predict(whole_X_test)
print(accuracy_score(whole_y_train, pred_train))
print(accuracy_score(whole_y_test, pred_test))

0.9740067877819925
0.9720330739299611


In [202]:
half_pred_train = half_clf.predict(X_train)
half_pred_test = half_clf.predict(X_test)

whole_pred_train = clf.predict(whole_X_train)
whole_pred_test = clf.predict(whole_X_test)

print(accuracy_score(y_train, half_pred_train))
print(accuracy_score(y_test, half_pred_test))

print(accuracy_score(whole_y_train, whole_pred_train))
print(accuracy_score(whole_y_test, whole_pred_test))


0.7999254843517138
0.6580937972768532
0.9771411459373128
0.9722357328145266


In [203]:
half_pred_train = half_clf.predict(whole_X_train)
half_pred_test = half_clf.predict(whole_X_test)

print(accuracy_score(whole_y_train, half_pred_train))
print(accuracy_score(whole_y_test, half_pred_test))

0.9001597125174685
0.8982247081712063
